# IDF値によるベクトライズの検討（part3）

## (1) TfidfVectorizer引数変更

プロダクションコードを以下のように修正し、nosetestsを流してみました。

ただし今回は、<b>「binary=True」というパラメータを追加し、TF値を強制的に 1 or 0 とする</b>よう指定しております。

- 修正前

> vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b')

- 修正後

> vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b', binary=True)

### (1-1) このパラメータの効果

binary=False時は、TF値が単語出現数の影響を受けますが、True時は全feature間でフラットな値になっています。

- use_idf=False, binary=False時

```
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ ＶＩＳＡ 取得 代金 勘定 科目']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 283)	0.353553390593
  (0, 404)	0.353553390593
  (0, 435)	0.353553390593
  (0, 865)	0.353553390593
  (0, 1181)	0.707106781187　<---TF値の影響が出てしまっています
learning.log: DEBUG: question: 「VISA」VISA取得のための代金はどの勘定科目なのか？

learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ 勘定 科目 わかる ない']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 80)	0.4472135955
  (0, 122)	0.4472135955
  (0, 404)	0.4472135955
  (0, 865)	0.4472135955
  (0, 1181)	0.4472135955
learning.log: DEBUG: question: VISAの勘定科目がわからない
```


- use_idf=False, binary=True時

```
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ ＶＩＳＡ 取得 代金 勘定 科目']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 283)	0.4472135955
  (0, 404)	0.4472135955
  (0, 435)	0.4472135955
  (0, 865)	0.4472135955
  (0, 1181)	0.4472135955　<---TF値の影響が回避できています
learning.log: DEBUG: question: 「VISA」VISA取得のための代金はどの勘定科目なのか？
：
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ＶＩＳＡ 勘定 科目 わかる ない']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 80)	0.4472135955
  (0, 122)	0.4472135955
  (0, 404)	0.4472135955
  (0, 865)	0.4472135955
  (0, 1181)	0.4472135955
learning.log: DEBUG: question: VISAの勘定科目がわからない
```

## (2) 結果の比較

### (2-1) accuracyの変化

全体的に大きな変化は見られませんでした。

| テストデータ | accuracy（改修前） | <> | accuracy（改修後） |
| :--- | :---: | :---: | :---: |
| test_daikin_conversation | 0.985786336543 | > | 0.984640073361 |
| test_ptna_conversation | 0.977212971078 | == | 0.977212971078 |
| test_toyotsu_human_conversation | 0.993685872139 | < | 0.995264404104 |
| test_benefitone_conversation | 0.979611650485 | < | 0.0.981553398058 |

### (2-2) probabilityの変化

こちらはテストケースごとに揺れが確認されております。

特にラベル０に対するprobaの低下が目立ちます。

またテストデータ「test_toyotsu_human_conversation」に対しては、ラベル6763のprobaがしきい値を下回るという副作用が確認されています。

| テストデータ | テストケース（質問文） | 正解ラベル | proba（改修前） | <> | proba（改修後） | 
| :--- |  :--- | :---: | :---: | :---: | :---: |
| test_daikin_conversation | しいたけは嫌いな食べ物です |0| 0.9816022499009649 | > | 0.94040053357176312 |
| test_daikin_conversation| 社外からのメール添付ファイルの受信容量制限は？ | 3487 | 0.8859353165274696 | < | 0.93266895642879499 |
| test_daikin_conversation | メールに署名を付ける方法を知りたい | 3398 | 0.89561349205672769 | < | 0.9426926063567419 |
| test_daikin_conversation | SPAMフォルダにメールが入っている | 3701 | 0.98016558676347543 | > | 0.92487661799934506 |
| test_ptna_conversation | (blank) | 0 | 0.64910771108380338 | > | 0.45396658057020628 |
| test_ptna_conversation | おいしいラーメンが食べたいです | 0 | 0.64910771108380338 | > | 0.45396658057020628 |
| test_ptna_conversation | こんにちは | 2684 | 0.60717161230416883 | > | 0.60575403165483244 |
| test_ptna_conversation | 女の先生 | 4068 | 0.98318553678782494 | < | 0.98800988116448674 |
| test_ptna_conversation | 入会したいのですが | 2688 | 0.90292395657147917 | > | 0.9009776260091249 |
| test_toyotsu_human_conversation | VISAの勘定科目がわからない | 6767 | 0.25251916431211241 | < | 0.58548209494611025 |
| test_toyotsu_human_conversation | JAL マイレージ | 6803 | 0.90142687232029783 | > | 0.86243381020685794 |
| test_toyotsu_human_conversation | 海外の出張費を精算したい | 6763 | 0.58534247570230802 | > | 0.44698777349025137 |
| test_benefitone_conversation | EXカードを貸してください | 4678 | 0.98849019014720063 | < | 0.99290874278286489 |
| test_benefitone_conversation | 契約書を見たいのですが | 4683 | 0.98824130181911307 | < | 0.99164983601570122 |

このため、プロダクションのテストケース<b>「test_toyotsu_human_conversation.py」がエラー</b>となってしまいました。

```
======================================================================
FAIL: test_overseas_business_trip_pay (learning.tests.engine.test_toyotsu_human_conversation.ToyotsuHumanConversationTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/engine/test_toyotsu_human_conversation.py", line 53, in test_overseas_business_trip_pay
    ok_(result.probability > self.threshold)
AssertionError: None
-------------------- >> begin captured logging << --------------------
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['海外 出張 費 精算 する']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 994)	0.54960468972
  (0, 886)	0.507202270577
  (0, 778)	0.441602245452
  (0, 376)	0.430631246213
  (0, 51)	0.245407270382
learning.log: DEBUG: question: 海外の出張費を精算したい
learning.log: DEBUG: question_feature_count: 5
learning.log: DEBUG: predicted results (order by probability desc)
learning.log: DEBUG: {'answer_id': 6763.0, 'probability': 0.44698777349025137}
learning.log: DEBUG: {'answer_id': 6856.0, 'probability': 0.25933482290817483}
learning.log: DEBUG: {'answer_id': 6762.0, 'probability': 0.099166998868494352}
learning.log: DEBUG: {'answer_id': 6830.0, 'probability': 0.070764538514641817}
learning.log: DEBUG: {'answer_id': 6847.0, 'probability': 0.022272903993338526}
learning.log: DEBUG: {'answer_id': 6827.0, 'probability': 0.010962494105183374}
learning.log: DEBUG: {'answer_id': 6840.0, 'probability': 0.008680119835088684}
learning.log: DEBUG: {'answer_id': 6802.0, 'probability': 0.0053639134843241187}
learning.log: DEBUG: {'answer_id': 6824.0, 'probability': 0.005111632818439845}
learning.log: DEBUG: {'answer_id': 6823.0, 'probability': 0.0045841130909014234}
--------------------- >> end captured logging << ---------------------```